In [41]:
import pandas as pd
import requests
import json
from geopy.distance import geodesic

In [42]:
# API 키와 URL 설정
api_key = "RZIkivnrKb8sF8J6at2jr4FvP0jBxaapaxY6Iuyf"
geo_url = "https://apis.openapi.sk.com/tmap/geo/fullAddrGeo"
route_url = "https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1&format=json"

In [43]:
headers = {
    "appKey": api_key,
    "Content-Type": "application/json"
}

# 출발지, 도착지 주소
start = "서울 마포구 백범로 35 서강대학교"
end = "서울 마포구 와우산로 94"

# 출발지 좌표 요청
start_params = {
    "version": 1,
    "fullAddr": start,
    "coordType": "WGS84GEO",
    "appKey": api_key
}
start_response = requests.get(geo_url, params=start_params)
start_data = start_response.json()
startX = start_data["coordinateInfo"]["coordinate"][0]["newLon"] #경도
startY = start_data["coordinateInfo"]["coordinate"][0]["newLat"] #위도 

# 도착지 좌표 요청
end_params = {
    "version": 1,
    "fullAddr": end,
    "coordType": "WGS84GEO",
    "appKey": api_key
}
end_response = requests.get(geo_url, params=end_params)
end_data = end_response.json()
endX = end_data["coordinateInfo"]["coordinate"][0]["newLon"] # 경도
endY = end_data["coordinateInfo"]["coordinate"][0]["newLat"] # 위도 

print(f"출발지 좌표: {startX}, {startY}")
print(f"도착지 좌표: {endX}, {endY}")

출발지 좌표: 126.943264, 37.550603
도착지 좌표: 126.92721, 37.551464


In [44]:
# 경로 탐색 요청 파라미터
route_params = {
    "startX": startX,
    "startY": startY,
    "endX": endX,
    "endY": endY,
    "reqCoordType": "WGS84GEO",
    "resCoordType": "EPSG3857",
    "startName": "출발지",
    "endName": "도착지",
}


# 경로 탐색 API 호출
response = requests.post(route_url, headers=headers, json=route_params)
if response.status_code == 200:
    data = response.json()
    
    # 총 거리, 시간
    total_distance = data["features"][0]["properties"]["totalDistance"]
    total_time_sec = data["features"][0]["properties"]["totalTime"]
    total_time_min = total_time_sec // 60 # 분
    total_time_sec = total_time_sec % 60 # 초
    print(f"총 거리: {total_distance} 미터")
    print(f"총 예상 시간: {total_time_min}분 {total_time_sec}초")

    # 각 구간별 세부 정보 (설명, 거리, 시간, 위도/경도)
    for feature in data["features"]:
        properties = feature["properties"]
        geometry = feature["geometry"]
        
        # 구간 설명, 거리 및 시간
        description = properties.get("description", "N/A")
        distance = properties.get("distance", None)
        time = properties.get("time", None)
        
        # 거리와 시간이 없을 경우 조건 처리
        distance_text = f"{distance}미터" if distance is not None else "정보 없음"
        
        # 시간 형식 변경: 60초 이상일 경우 분과 초로 변환
        if time is not None:
            if time >= 60:
                minutes = time // 60
                seconds = time % 60
                time_text = f"{minutes}분 {seconds}초"
            else:
                time_text = f"{time}초"
        else:
            time_text = "정보 없음"
        
        # 구간 위도와 경도 (모든 좌표 포함)
        coordinates = []
        if geometry["type"] == "Point":
            coordinates.append(f"({geometry['coordinates'][1]}, {geometry['coordinates'][0]})")
        elif geometry["type"] == "LineString":
            for coord in geometry["coordinates"]:
                coordinates.append(f"({coord[1]}, {coord[0]})")
        
        # 출력 형식: 구간 설명, 거리, 시간, 모든 좌표
        coordinates_text = ", ".join(coordinates)
        print(f"구간 설명: {description}, 거리: {distance_text}, 예상 시간: {time_text}, 위치: {coordinates_text}")
else:
    print("API 요청 실패:", response.status_code)

총 거리: 2121 미터
총 예상 시간: 28분 2초
구간 설명: 보행자도로 을 따라 74m 이동, 거리: 정보 없음, 예상 시간: 정보 없음, 위치: (4516153, 14131251)
구간 설명: 보행자도로, 74m, 거리: 74미터, 예상 시간: 1분 1초, 위치: (4516153, 14131251), (4516166, 14131281), (4516157, 14131340)
구간 설명: 우회전 후 51m 이동 , 거리: 정보 없음, 예상 시간: 정보 없음, 위치: (4516157, 14131340)
구간 설명: , 51m, 거리: 51미터, 예상 시간: 36초, 위치: (4516157, 14131340), (4516095, 14131323)
구간 설명: 우회전 후 대흥로 을 따라 24m 이동 , 거리: 정보 없음, 예상 시간: 정보 없음, 위치: (4516095, 14131323)
구간 설명: 대흥로, 24m, 거리: 24미터, 예상 시간: 18초, 위치: (4516095, 14131323), (4516080, 14131305), (4516075, 14131299)
구간 설명: , 400m, 거리: 400미터, 예상 시간: 4분 46초, 위치: (4516075, 14131299), (4516074, 14131297), (4516055, 14131228), (4516048, 14131199), (4516047, 14131197), (4516046, 14131191), (4516040, 14131165), (4516028, 14131121), (4516005, 14131030), (4515986, 14130954), (4515982, 14130938), (4515975, 14130911), (4515973, 14130904), (4515972, 14130899), (4515963, 14130861), (4515957, 14130834), (4515957, 14130832), (4515956, 14130811)
구간 설명: 우회전 후 백범로 을 따라 107m 

In [45]:
# 출발지와 도착지 위도, 경도 저장
coordinates = {
    "출발지": (startY, startX),
    "도착지": (endY, endX),
    "구간": []
}

# JSON 결과에서 각 구간의 위도와 경도 추출
for feature in data["features"]:
    geometry = feature["geometry"]

    # Point 타입일 경우 해당 좌표를 추가
    if geometry["type"] == "Point":
        lat, lon = geometry["coordinates"][1], geometry["coordinates"][0]
        coordinates["구간"].append((lat, lon))
    
    # LineString 타입일 경우 모든 좌표를 추가
    elif geometry["type"] == "LineString":
        for coord in geometry["coordinates"]:
            lat, lon = coord[1], coord[0]
            coordinates["구간"].append((lat, lon))

# 최종 위도와 경도만 출력
print("출발지:", coordinates["출발지"])
print("도착지:", coordinates["도착지"])
print("구간 좌표:")
for latlon in coordinates["구간"]:
    print(latlon)


출발지: ('37.550603', '126.943264')
도착지: ('37.551464', '126.92721')
구간 좌표:
(4516153, 14131251)
(4516153, 14131251)
(4516166, 14131281)
(4516157, 14131340)
(4516157, 14131340)
(4516157, 14131340)
(4516095, 14131323)
(4516095, 14131323)
(4516095, 14131323)
(4516080, 14131305)
(4516075, 14131299)
(4516075, 14131299)
(4516074, 14131297)
(4516055, 14131228)
(4516048, 14131199)
(4516047, 14131197)
(4516046, 14131191)
(4516040, 14131165)
(4516028, 14131121)
(4516005, 14131030)
(4515986, 14130954)
(4515982, 14130938)
(4515975, 14130911)
(4515973, 14130904)
(4515972, 14130899)
(4515963, 14130861)
(4515957, 14130834)
(4515957, 14130832)
(4515956, 14130811)
(4515956, 14130811)
(4515956, 14130811)
(4515956, 14130794)
(4515964, 14130787)
(4515978, 14130773)
(4515996, 14130760)
(4516012, 14130750)
(4516044, 14130735)
(4516048, 14130734)
(4516055, 14130730)
(4516055, 14130730)
(4516055, 14130730)
(4516041, 14130698)
(4516041, 14130698)
(4516041, 14130698)
(4516034, 14130701)
(4516034, 14130701)
(4516033